# Исследование надежности заемщиков

### Шаг 1. Обзор данных

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 
Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Таблица содержит 12 столбцов и 21525 строк c типом данных в скобках:
    
* children — количество детей в семье (int64)
* days_employed — общий трудовой стаж в днях (float64)
* dob_years — возраст клиента в годах (int64)
* education — уровень образования клиента (object)
* education_id — идентификатор уровня образования (int64)
* family_status — семейное положение (object)
* family_status_id — идентификатор семейного положения (int64)
* gender — пол клиента (object)
* income_type — тип занятости (object)
* debt — имел ли задолженность по возврату кредитов (int64)
* total_income — ежемесячный доход (float64)
* purpose — цель получения кредита (object)

Количество значения в столбцах days_employed и total_income говорит о том, что есть пропуски

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/datasets/data.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Шаг 2.1 Заполнение пропусков

1. В столбцах days_employed и total_income были обнаружены пропущенные значения типа float64. 
2. Доля пропущенных значений в каждом из столбцов с пропусками составляет около 10% (количество пропусков в столбцах составляет 2174, получаем 2174 * 100 / 21525 = 10,0998..) 
3. Возможными причинами появления пропуска в столбце days_employed является отсутствие трудового стажа, что могло бы означать, что человек либо не работает, либо работает не официально, поэтому пропуски и в столбце total_income.
4. Пропущенные значения лучше заполнить медианным значением, т.к в выборке присутствуют абсолютно разные числовые значения.

In [4]:
days_employed_median = df['days_employed'].median() # Находим медианное значение
total_income_median = df['total_income'].median()
df['days_employed'] = df['days_employed'].fillna(days_employed_median) # Заменяем пропуски
df['total_income'] = df['total_income'].fillna(total_income_median)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Шаг 2.2 Проверка данных на аномалии и исправления.

In [5]:
display(df['children'].unique()) # Есть минусовые значения, возможно неправильно занесли данные
df['children'] = df['children'].replace(-1,1) # Заменяем значение в столбце children -1 на 1 
df['children'] = df['children'].replace(20,2) # Заменяем значение в столбце children 20 на 2
display(df['children'].unique()) # Проверка результата

array([ 1,  0,  3,  2, -1,  4, 20,  5])

array([1, 0, 3, 2, 4, 5])

In [6]:
display(df['days_employed'].unique()) # Все минусовые значения, возможна причина - при заполнении произошла  техническая ошибка
df['days_employed'] = df['days_employed'].abs() # Заменяем отрицательные значения методом abs
display(df['days_employed'].unique()) # Проверка результата

array([-8437.67302776, -4024.80375385, -5623.42261023, ...,
       -2113.3468877 , -3112.4817052 , -1984.50758853])

array([8437.67302776, 4024.80375385, 5623.42261023, ..., 2113.3468877 ,
       3112.4817052 , 1984.50758853])

In [7]:
 print(df[df['dob_years'] == 0]['dob_years'].count()) # Подсчет количества записей в колонке dob_years со значением 0 

101


Поскольку количество записей в колонке dob_years со значением 0 является 101, логично заменить его на медиану. 

In [8]:
print('Медиана в dob_years: ',df['dob_years'].median())
df['dob_years'] = df['dob_years'].replace(0, df['dob_years'].median())

Медиана в dob_years:  42.0


In [9]:
 print(df[df['dob_years'] == 0]['dob_years'].count()) # Подсчет количества записей в колонке dob_years со значением 0 после изменения

0


Нужно привести к одному регистру и есть дубликаты, возможной причиной стал человеческий фактор.
Удаление дубликатов происходит в "0.5  Шаг 2.4. Удаление дубликатов."

In [10]:
display(df['education'].unique())

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [11]:
display(df['education_id'].unique()) # Все ОК

array([0, 1, 2, 3, 4])

In [12]:
display(df['family_status'].unique()) # Все ОК

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [13]:
display(df['family_status_id'].unique()) # Все ОК

array([0, 1, 2, 3, 4])

In [14]:
display(df['gender'].unique()) # XNA, ошибочно написали
df = df.drop(df[df['gender'] == 'XNA'].index) # Удаляем в столбце gender значение XNA
display(df['gender'].unique()) # Проверка результата

array(['F', 'M', 'XNA'], dtype=object)

array(['F', 'M'], dtype=object)

In [15]:
display(df['income_type'].unique()) # Все ОК

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [16]:
display(df['debt'].unique()) # Все ОК

array([0, 1])

In [17]:
display(df['total_income'].unique()) # Все ОК

array([253875.6394526 , 112080.01410244, 145885.95229686, ...,
        89672.56115303, 244093.05050043,  82047.41889948])

In [18]:
display(df['purpose'].unique()) # Неявные дубликаты. Категоризированно по ключевым словам в п. "0.8  Шаг 2.7. Категоризация целей кредита."

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

### Шаг 2.3. Изменение типов данных.

In [19]:
try:
    # Замена вещественного типа на целоцисленный
    df['total_income']= df['total_income'].astype('int') 
    df['days_employed'] = df['days_employed'].astype('int')
except Exception as e:
    print('Не удалось преобразовать поле total_income в тип int')

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21524 non-null  int64  
 1   days_employed     21524 non-null  int64  
 2   dob_years         21524 non-null  float64
 3   education         21524 non-null  object 
 4   education_id      21524 non-null  int64  
 5   family_status     21524 non-null  object 
 6   family_status_id  21524 non-null  int64  
 7   gender            21524 non-null  object 
 8   income_type       21524 non-null  object 
 9   debt              21524 non-null  int64  
 10  total_income      21524 non-null  int64  
 11  purpose           21524 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.1+ MB


### Шаг 2.4. Удаление дубликатов.

In [21]:
df['education'].value_counts() # Подсчет количества До

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      667
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [22]:
df['education'] = df['education'].str.lower() # Приводим к нижнему регистру

In [23]:
df['education'].value_counts() # Подсчет количества После

среднее                15233
высшее                  5260
неоконченное высшее      743
начальное                282
ученая степень             6
Name: education, dtype: int64

In [24]:
df.duplicated().sum() # Поиск дубликатов во всем датафрейме до устранения

72

In [25]:
df = df.drop_duplicates().reset_index(drop = True)

In [26]:
df.duplicated().sum() # Поиск дубликатов во всем датафрейме после устранения

0

1. Дубликаты в поле education появились из-за несоблюдения регистров при заполнении данных. Для устранения дубликатов все строки столбца education были приведены к нижнему регистру.
2. Во всем датафрейме было 71 дублей, они тоже были устранены методом drop_duplicates.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [27]:
print(df['education'].unique()) # Вывод уникальных значений
print(df['education_id'].unique())

['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']
[0 1 2 3 4]


In [28]:
print(df['family_status'].unique())
print(df['family_status_id'].unique())

['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']
[0 1 2 3 4]


In [29]:
try:
    del df['education']
    del df['family_status']
except Exception as e:
    print('Произошла ошибка при удалении полей education и family_status')

In [30]:
education_name = ['высшее', 'среднее', 'неоконченное высшее', 'начальное', 'ученая степень']
education_id = [0, 1, 2, 3, 4]
df_education = pd.DataFrame({'education_id':education_id, 'education_name': education_name}) # Формирование словаря образований
display(df_education)

,education_id,education_name
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [31]:
family_status_name = ['женат / замужем', 'гражданский брак', 'вдовец / вдова', 'в разводе', 'Не женат / не замужем']
family_status_id = [0, 1, 2, 3, 4]
df_family_status = pd.DataFrame({'family_status_id': family_status_id,'family_status_name': family_status_name}) # Формирование словаря семейного положения
display(df_family_status)

,family_status_id,family_status_name
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


### Шаг 2.6. Категоризация дохода.

In [32]:
def get_total_income_category(total_income): # Создаем функцию
    if total_income <= 30000:
        return 'E'
    if total_income <= 50000:
        return 'D'
    if total_income <= 200000:
        return 'C'
    if total_income <= 1000000:
        return 'B'
    return 'A'

In [33]:
get_total_income_category(25000) # Проверяем работу функции при разных значениях на входе

'E'

In [34]:
get_total_income_category(235000) # Проверяем работу функции при разных значениях на входе

'B'

In [35]:
df['total_income_category'] = df['total_income'].apply(get_total_income_category) # Создаем столбец

### Шаг 2.7. Категоризация целей кредита.

In [36]:
display(df['purpose'].unique()) # Неявные дубликаты, причина - человеческий фактор

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [37]:
def get_purpose_category(purpose): # Создаем функцию
    if 'автомобил' in purpose:
        return 'операции с автомобилем'
    if ('жиль' in purpose) or ('недвижимост' in purpose):
        return 'операции с недвижимостью'
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'образовани' in purpose:
        return 'получение образования'

In [38]:
get_purpose_category('на покупку автомобиля') # Проверяем работу функции при разных значениях на входе

'операции с автомобилем'

In [39]:
df['purpose_category'] = df['purpose'].apply(get_purpose_category) # Проверяем работу функции при разных значениях на входе

In [40]:
df['purpose_category'].value_counts() # Подсчет уникальных значений 

операции с недвижимостью    10810
операции с автомобилем       4306
получение образования        4013
проведение свадьбы           2323
Name: purpose_category, dtype: int64

### Ответы на вопросы.

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [41]:
# Вычисляем количество выдачи кредитов в зависимости от детей в семье
children_count_1 = pd.pivot_table(df, index='children', values='debt', aggfunc='count')
# Вычисляем задолжности по кредитам в зависимости от детей в семье
children_sum_1 = pd.pivot_table(df, index='children', values='debt', aggfunc='sum') 
# Вычисляем процентное соотношение
quality_children = children_sum_1 / children_count_1 * 100
# Вывод полученного датафрейма
display(quality_children)

,debt
children,
0,7.544893
1,9.165808
2,9.492481
3,8.181818
4,9.756098
5,0.000000


In [42]:
# Альтернатива с mean
children_mean = pd.pivot_table(df, index='children', values='debt', aggfunc='mean')
children_mean = children_mean * 100
display(children_mean)

,debt
children,
0,7.544893
1,9.165808
2,9.492481
3,8.181818
4,9.756098
5,0.000000


##### Вывод 1:

Больше всего задолженностей у семей с детьми от 1-4 детей. У семей с 5 детьми нету задолженностей. Это может быть связано с финансами.

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [43]:
family_count = pd.pivot_table(df, index='family_status_id', values='debt', aggfunc='count')
family_sum = pd.pivot_table(df, index='family_status_id', values='debt', aggfunc='sum')
quality_family = family_sum / family_count * 100
display(df_family_status) # Справочник семейного положения
display(quality_family)

,family_status_id,family_status_name
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


,debt
family_status_id,
0,7.545182
1,9.351651
2,6.569343
3,7.112971
4,9.750890


In [44]:
# Вариант с merge
mergerd_quality_family = quality_family.merge(df_family_status, on='family_status_id', how='left')
display(mergerd_quality_family)

,family_status_id,debt,family_status_name
0,0,7.545182,женат / замужем
1,1,9.351651,гражданский брак
2,2,6.569343,вдовец / вдова
3,3,7.112971,в разводе
4,4,9.750890,Не женат / не замужем


##### Вывод 2:

У категорий "не женат / не замужем" и "гражданский брак" больше всего задолжностей, можно судить, что они не хотят нести ответственность ни за финансы, ни в жизни, а в категории  "вдовец / вдова" и "в разводе" меньше всего задолжностей, эти люди остались одни, пережили многое и чувство ответственности у них большое. В категории "женат / замужем" средние показатели, вероятнее всего эти люди ответственны, но есть не большие проблемы с финансами.

##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [45]:
total_income_count = pd.pivot_table(df, index='total_income_category', values='debt', aggfunc='count')
total_income_sum = pd.pivot_table(df, index='total_income_category', values='debt', aggfunc='sum')
quality_total_income = total_income_sum / total_income_count * 100
display(quality_total_income)

,debt
total_income_category,
A,8.000000
B,7.063492
C,8.492039
D,6.000000
E,9.090909


##### Вывод 3:

1. Клиенты, уровень дохода которых 30 000 – 50 000 руб., они умеют рассчитывать свои возможности и траты
2. Клиенты, уровень дохода которых больше 50 000 руб., задолжностей больше вероятнее всего потому что и траты у них больше. 
3. Клиенты, у которых доход составляет до 30 000 руб., у них больше всех задолжностей, вероятно, это связано с низким уровнем дохода.

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

In [46]:
purpose_count = pd.pivot_table(df, index='purpose_category', values='debt', aggfunc='count')
purpose_sum = pd.pivot_table(df, index='purpose_category', values='debt', aggfunc='sum')
quality_purpose = purpose_sum / purpose_count * 100
display(quality_purpose)

,debt
purpose_category,
операции с автомобилем,9.359034
операции с недвижимостью,7.234043
получение образования,9.220035
проведение свадьбы,8.006888


##### Вывод 4:

1. У категорий "операции с недвижимостью" и "проведение свадьбы" меньше всего задолжностей, к недвижимости люди относятся ответственно, т.к срок платежа большой и не хочется остаться в долгах на крупные суммы, это пугает людей. А после свадьбы гости дарят деньги, которые потом можно положить на кредит, поэтому должников меньше.
2. У категорий "операции с автомобилем" и "получение образования" задолжностей больше, может быть потому что люди берут кредит не совсем обдуманно, например поведясь на рекламу машины. Одновременно учиться и работать тяжело, и найти работу с хорошей заработной платой трудно, появляются задолжности из-за недостачи финансов.

## Общий вывод:

По итогам своей работы, могу сказать, что самыми надежными клиентами будут люди, имеющие 5 детей (т.к. они более опытные и ответственные), вдовец или вдова, имеющие уровень дохода 30 000 – 50 000 руб в месяц, которые берут кредит для операции с недвижимостью.

Не надежными кредиторами будут люди, имеющие 1, 2, 4 детей, одиночки, у которых доход составляет до 30 000 руб и целью является покупка автомобиля.